# MoneyTrend

In [1]:
#2019-01-28 수정
# 기존 daum 주식 사이트 : ajax 방식으로 변경으로 인해 이를 반영한 코드를 수정.
# pip install fake-useragent 설치 후 실행 가능

import io
import json
import sys
import pandas as pd
import urllib.request as req

from fake_useragent import UserAgent

#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding='utf-8')

# Fake Header 정보
ua = UserAgent()

# 헤더 선언
headers = {
    'User-Agent': ua.ie,
    'referer': 'https://finance.daum.net/'
}



# 다음 주식 요청 URL
df1 = pd.DataFrame()
for i in range(1,3):
    url = "https://finance.daum.net/api/trend/arounds?page="+str(i)+"&perPage=20&fieldName=changeRate&order=desc&pagination=true"
    res = req.urlopen(req.Request(url, headers=headers)).read().decode('utf-8')
    
    rank_json = json.loads(res)['data']
    
    df = pd.DataFrame(rank_json)
    
    df1 = df1.append(df)
    #print('중간 확인 : ', kkang, '\n')
    #print(url)
#print(kkang)
#df = pd.DataFrame(kkang)
df1

,beneficiaryBond,beneficiaryBondChange,beneficiaryMix,beneficiaryMixChange,beneficiaryStock,beneficiaryStockChange,date,depositPrice,depositPriceChange,realDepositPrice,realDepositPriceChange,receivablePrice,receivablePriceChange
0,108378600,291200,27521200,24400,85510400,12000,2019-03-27,26550166,534697,16170738,600282,148720,14178
1,108087400,-600,27496800,-900,85498400,146300,2019-03-26,26015469,-684950,15570456,-938828,134542,-6974
2,108088000,193200,27497700,-39300,85352100,-976200,2019-03-25,26700419,1404296,16509284,1646269,141516,5922
3,107894800,-235500,27537000,-34500,86328300,-1400,2019-03-22,25296123,832228,14863015,1156627,135594,-9097
4,108130300,168600,27571500,6800,86329700,156800,2019-03-21,24463895,-365322,13706388,-813948,144691,11387
5,107961700,5500,27564700,8900,86172900,2800,2019-03-20,24829217,448403,14520336,446521,133304,14028
6,107956200,33600,27555800,-6700,86170100,70100,2019-03-19,24380814,-9010,14073815,448276,119276,3603
7,107922600,200900,27562500,1900,86100000,-109600,2019-03-18,24389824,358820,13625539,333788,115673,1558
8,107721700,88300,27560600,-2300,86209600,144700,2019-03-15,24031004,-768279,13291751,-1176244,114115,4483
9,107633400,321300,27562900,7300,86064900,-47500,2019-03-14,24799283,-10024,14467995,2793,109632,-5820


# 선물

In [3]:
# 2019-01-28 수정
# 기존 daum 주식 사이트 : ajax 방식으로 변경으로 인해 이를 반영한 코드를 수정.
# pip install fake-useragent 설치 후 실행 가능

import io
import json
import sys
import pandas as pd
import urllib.request as req

from fake_useragent import UserAgent

#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding='utf-8')

# Fake Header 정보
ua = UserAgent()

# 헤더 선언
headers = {
    'User-Agent': ua.ie,
    'referer': 'https://finance.daum.net/domestic/futures'
}


df1 = pd.DataFrame()
for i in range(1,3):
    # 다음 주식 요청 URL
    url = "https://finance.daum.net/api/future/KR4101P60004/days?pagination=true&page="+str(i)

    # 요청
    res = req.urlopen(req.Request(url, headers=headers)).read().decode('utf-8')

    df = pd.DataFrame(rank_json)
    
    df1 = df1.append(df)
    #print('중간 확인 : ', kkang, '\n')
    #print(url)
#print(kkang)
#df = pd.DataFrame(kkang)
df1

,accTradeVolume,change,changePrice,changeRate,date,foreignSettlement,institutionSettlement,privateSettlement,tradePrice,unsettledVolume
0,960,FALL,4.50,0.0156,2019-02-28 00:00:00,1230,-1839,1196,284.50,26493
1,436,RISE,0.50,0.0017,2019-02-27 00:00:00,-703,-1289,2706,289.00,26116
2,343,FALL,1.65,0.0057,2019-02-26 00:00:00,1142,482,-1580,288.50,26132
3,515,RISE,0.80,0.0028,2019-02-25 00:00:00,-5836,3242,2819,290.15,26156
4,290,RISE,0.25,0.0009,2019-02-22 00:00:00,2811,-1274,-1622,289.35,25811
5,427,RISE,0.15,0.0005,2019-02-21 00:00:00,-3570,3098,263,289.10,25737
6,382,RISE,2.85,0.0100,2019-02-20 00:00:00,-4587,2374,2920,288.95,25463
7,250,FALL,1.20,0.0042,2019-02-19 00:00:00,-1168,97,448,286.10,24807
8,350,RISE,2.50,0.0088,2019-02-18 00:00:00,-1454,-1060,2981,287.30,24214
9,452,FALL,4.05,0.0140,2019-02-15 00:00:00,3043,-1601,-1451,284.80,22623


In [13]:

# 2019-01-28 수정
# 기존 daum 주식 사이트 : ajax 방식으로 변경으로 인해 이를 반영한 코드를 수정.
# pip install fake-useragent 설치 후 실행 가능

import io
import json
import sys
import pandas as pd
import urllib.request as req

from fake_useragent import UserAgent

#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding='utf-8')

# Fake Header 정보
ua = UserAgent()

# 헤더 선언
headers = {
    'User-Agent': ua.ie,
    'referer': 'https://finance.daum.net/'
}

# 다음 주식 요청 URL
url = "https://finance.daum.net/api/search/ranks?limit=10"

# print(request.get_method())   #Post or Get 확인
# print(request.get_full_url()) #요청 Full Url 확인

# 요청
res = req.urlopen(req.Request(url, headers=headers)).read().decode('utf-8')

# 응답 데이터 확인(Json Data)
# print('res', res)

# 응답 데이터 str -> json 변환 및 data 값 저장
rank_json = json.loads(res)['data']

# 중간 확인
print('중간 확인 : ', rank_json, '\n')

#for elm in rank_json:
    # print(type(elm)) #Type 확인
    #print('순위 : {}, 금액 : {}, 회사명 : {}'.format(elm['rank'], elm['tradePrice'], elm['name']), )

df = pd.DataFrame(rank_json)
df

중간 확인 :  [{'changePrice': 2500, 'tradePrice': 180500, 'rank': 1, 'code': 'KR7068270008', 'changeRate': 0.0136612022, 'name': '셀트리온', 'isNew': False, 'symbolCode': 'A068270', 'change': 'FALL', 'chartSlideImage': None, 'rankChange': 0}, {'changePrice': 13200, 'tradePrice': 88700, 'rank': 2, 'code': 'KR7307950006', 'changeRate': 0.1748344371, 'name': '현대오토에버', 'isNew': False, 'symbolCode': 'A307950', 'change': 'RISE', 'chartSlideImage': None, 'rankChange': 0}, {'changePrice': 100, 'tradePrice': 3520, 'rank': 3, 'code': 'KR7020560009', 'changeRate': 0.0292397661, 'name': '아시아나항공', 'isNew': True, 'symbolCode': 'A020560', 'change': 'RISE', 'chartSlideImage': None, 'rankChange': 0}, {'changePrice': 130, 'tradePrice': 9610, 'rank': 4, 'code': 'KR7048410005', 'changeRate': 0.0137130802, 'name': '현대바이오', 'isNew': False, 'symbolCode': 'A048410', 'change': 'RISE', 'chartSlideImage': None, 'rankChange': 1}, {'changePrice': 500, 'tradePrice': 44850, 'rank': 5, 'code': 'KR7005930003', 'changeRate': 0

,change,changePrice,changeRate,chartSlideImage,code,isNew,name,rank,rankChange,symbolCode,tradePrice
0,FALL,2500,0.013661,None,KR7068270008,False,셀트리온,1,0,A068270,180500
1,RISE,13200,0.174834,None,KR7307950006,False,현대오토에버,2,0,A307950,88700
2,RISE,100,0.029240,None,KR7020560009,True,아시아나항공,3,0,A020560,3520
3,RISE,130,0.013713,None,KR7048410005,False,현대바이오,4,1,A048410,9610
4,FALL,500,0.011025,None,KR7005930003,False,삼성전자,5,-2,A005930,44850
5,FALL,1400,0.021705,None,KR7215600008,False,신라젠,6,2,A215600,63100
6,RISE,55,0.027569,None,USU652221081,False,뉴프라이드,7,-3,A900100,2050
7,FALL,1750,0.052711,None,KR7003490000,False,대한항공,8,2,A003490,31450
8,EVEN,0,0.000000,None,KR7005690003,False,파미셀,9,0,A005690,11900
9,FALL,1950,0.050129,None,KR7020150009,True,일진머티리얼즈,10,0,A020150,36950


In [57]:
# 2019-01-28 수정
# 기존 daum 주식 사이트 : ajax 방식으로 변경으로 인해 이를 반영한 코드를 수정.
# pip install fake-useragent 설치 후 실행 가능

import io
import json
import sys
import pandas as pd
import urllib.request as req

from fake_useragent import UserAgent

#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding='utf-8')

# Fake Header 정보
ua = UserAgent()

# 헤더 선언
headers = {
    'User-Agent': ua.ie,
    'referer': 'https://finance.daum.net/'
}

# 다음 주식 요청 URL
url = "https://finance.daum.net/api/trend/arounds?page=1&perPage=20&fieldName=changeRate&order=desc&pagination=true"

# print(request.get_method())   #Post or Get 확인
# print(request.get_full_url()) #요청 Full Url 확인

# 요청
res = req.urlopen(req.Request(url, headers=headers)).read().decode('utf-8')

# 응답 데이터 확인(Json Data)
# print('res', res)

# 응답 데이터 str -> json 변환 및 data 값 저장
rank_json = json.loads(res)['data']

# 중간 확인
print('중간 확인 : ', rank_json, '\n')

#for elm in rank_json:
    # print(type(elm)) #Type 확인
    #print('순위 : {}, 금액 : {}, 회사명 : {}'.format(elm['rank'], elm['tradePrice'], elm['name']), )

df = pd.DataFrame(rank_json)
df

중간 확인 :  [{'realDepositPrice': 15570456, 'receivablePrice': 134542, 'depositPrice': 26015469, 'beneficiaryMix': 27496800, 'beneficiaryBond': 108087400, 'beneficiaryStockChange': 146300, 'receivablePriceChange': -6974, 'beneficiaryBondChange': -600, 'beneficiaryStock': 85498400, 'realDepositPriceChange': -938828, 'depositPriceChange': -684950, 'beneficiaryMixChange': -900, 'date': '2019-03-26'}, {'realDepositPrice': 16509284, 'receivablePrice': 141516, 'depositPrice': 26700419, 'beneficiaryMix': 27497700, 'beneficiaryBond': 108088000, 'beneficiaryStockChange': -976200, 'receivablePriceChange': 5922, 'beneficiaryBondChange': 193200, 'beneficiaryStock': 85352100, 'realDepositPriceChange': 1646269, 'depositPriceChange': 1404296, 'beneficiaryMixChange': -39300, 'date': '2019-03-25'}, {'realDepositPrice': 14863015, 'receivablePrice': 135594, 'depositPrice': 25296123, 'beneficiaryMix': 27537000, 'beneficiaryBond': 107894800, 'beneficiaryStockChange': -1400, 'receivablePriceChange': -9097, 'be

,beneficiaryBond,beneficiaryBondChange,beneficiaryMix,beneficiaryMixChange,beneficiaryStock,beneficiaryStockChange,date,depositPrice,depositPriceChange,realDepositPrice,realDepositPriceChange,receivablePrice,receivablePriceChange
0,108087400,-600,27496800,-900,85498400,146300,2019-03-26,26015469,-684950,15570456,-938828,134542,-6974
1,108088000,193200,27497700,-39300,85352100,-976200,2019-03-25,26700419,1404296,16509284,1646269,141516,5922
2,107894800,-235500,27537000,-34500,86328300,-1400,2019-03-22,25296123,832228,14863015,1156627,135594,-9097
3,108130300,168600,27571500,6800,86329700,156800,2019-03-21,24463895,-365322,13706388,-813948,144691,11387
4,107961700,5500,27564700,8900,86172900,2800,2019-03-20,24829217,448403,14520336,446521,133304,14028
5,107956200,33600,27555800,-6700,86170100,70100,2019-03-19,24380814,-9010,14073815,448276,119276,3603
6,107922600,200900,27562500,1900,86100000,-109600,2019-03-18,24389824,358820,13625539,333788,115673,1558
7,107721700,88300,27560600,-2300,86209600,144700,2019-03-15,24031004,-768279,13291751,-1176244,114115,4483
8,107633400,321300,27562900,7300,86064900,-47500,2019-03-14,24799283,-10024,14467995,2793,109632,-5820
9,107312100,19800,27555600,-1300,86112400,-341100,2019-03-13,24809307,688450,14465202,1144114,115452,-1801


In [80]:
#2019-01-28 수정
# 기존 daum 주식 사이트 : ajax 방식으로 변경으로 인해 이를 반영한 코드를 수정.
# pip install fake-useragent 설치 후 실행 가능

import io
import json
import sys
import pandas as pd
import urllib.request as req

from fake_useragent import UserAgent

#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding='utf-8')

# Fake Header 정보
ua = UserAgent()

# 헤더 선언
headers = {
    'User-Agent': ua.ie,
    'referer': 'https://finance.daum.net/'
}



# 다음 주식 요청 URL
df1 = pd.DataFrame()
for i in range(1,3):
    url = "https://finance.daum.net/api/trend/arounds?page="+str(i)+"&perPage=20&fieldName=changeRate&order=desc&pagination=true"
    res = req.urlopen(req.Request(url, headers=headers)).read().decode('utf-8')
    
    rank_json = json.loads(res)['data']
    
    df = pd.DataFrame(rank_json)
    
    df1 = df1.append(df)
    #print('중간 확인 : ', kkang, '\n')
    #print(url)
#print(kkang)
#df = pd.DataFrame(kkang)
df1

,beneficiaryBond,beneficiaryBondChange,beneficiaryMix,beneficiaryMixChange,beneficiaryStock,beneficiaryStockChange,date,depositPrice,depositPriceChange,realDepositPrice,realDepositPriceChange,receivablePrice,receivablePriceChange
0,108087400,-600,27496800,-900,85498400,146300,2019-03-26,26015469,-684950,15570456,-938828,134542,-6974
1,108088000,193200,27497700,-39300,85352100,-976200,2019-03-25,26700419,1404296,16509284,1646269,141516,5922
2,107894800,-235500,27537000,-34500,86328300,-1400,2019-03-22,25296123,832228,14863015,1156627,135594,-9097
3,108130300,168600,27571500,6800,86329700,156800,2019-03-21,24463895,-365322,13706388,-813948,144691,11387
4,107961700,5500,27564700,8900,86172900,2800,2019-03-20,24829217,448403,14520336,446521,133304,14028
5,107956200,33600,27555800,-6700,86170100,70100,2019-03-19,24380814,-9010,14073815,448276,119276,3603
6,107922600,200900,27562500,1900,86100000,-109600,2019-03-18,24389824,358820,13625539,333788,115673,1558
7,107721700,88300,27560600,-2300,86209600,144700,2019-03-15,24031004,-768279,13291751,-1176244,114115,4483
8,107633400,321300,27562900,7300,86064900,-47500,2019-03-14,24799283,-10024,14467995,2793,109632,-5820
9,107312100,19800,27555600,-1300,86112400,-341100,2019-03-13,24809307,688450,14465202,1144114,115452,-1801


In [74]:
df.shape

(1, 20)

In [50]:
#2019-01-28 수정
# 기존 daum 주식 사이트 : ajax 방식으로 변경으로 인해 이를 반영한 코드를 수정.
# pip install fake-useragent 설치 후 실행 가능

import io
import json
import sys
import pandas as pd
import urllib.request as req

from fake_useragent import UserAgent

#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding='utf-8')

# Fake Header 정보
ua = UserAgent()

# 헤더 선언
headers = {
    'User-Agent': ua.ie,
    'referer': 'https://finance.daum.net/'
}

# 다음 주식 요청 URL
for i in range(1,3):
    url = "https://finance.daum.net/api/trend/arounds?page="+str(i)+"&perPage=20&fieldName=changeRate&order=desc&pagination=true"
    print(url)

https://finance.daum.net/api/trend/arounds?page=1&perPage=20&fieldName=changeRate&order=desc&pagination=true
https://finance.daum.net/api/trend/arounds?page=2&perPage=20&fieldName=changeRate&order=desc&pagination=true


In [ ]:
# 2019-01-28 수정
# 기존 daum 주식 사이트 : ajax 방식으로 변경으로 인해 이를 반영한 코드를 수정.
# pip install fake-useragent 설치 후 실행 가능

import io
import json
import sys
import pandas as pd
import urllib.request as req

from fake_useragent import UserAgent

#sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')
#sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding='utf-8')

# Fake Header 정보
ua = UserAgent()

# 헤더 선언
headers = {
    'User-Agent': ua.ie,
    'referer': 'https://finance.daum.net/'
}

# 다음 주식 요청 URL
for i in range(1,3):
    url = "https://finance.daum.net/api/trend/arounds?page="+str(i)+"&perPage=20&fieldName=changeRate&order=desc&pagination=true"
    print(url)
    res = req.urlopen(req.Request(url, headers=headers)).read().decode('utf-8')

    rank_json = json.loads(res)['data']
    print('중간 확인 : ', rank_json, '\n')

    #df = pd.DataFrame(rank_json)
    #df


In [26]:
import FinanceDataReader as fdr
import pandas as pd
from bs4 import BeautifulSoup
import datetime as dt
from datetime import datetime,timedelta
from urllib.request import urlopen
import sqlalchemy 
import pymysql
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

url = 'https://finance.daum.net/domestic/arounds'
source = urlopen(url).read()   # 지정한 페이지에서 코드 읽기
source = BeautifulSoup(source, 'lxml')   # 뷰티풀 스프로 태그별로 코드 분류
print(source)


<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="//finance.daum.net/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<title>증시주변자금동향 | 다음 금융 </title>
<link href="/dist/common.css?v=1552461157" rel="stylesheet" type="text/css"/>
<link href="/dist/custom.ui.css?v=1552461157" rel="stylesheet" type="text/css"/>
<script>
    window.REQUEST_URI = '/domestic/arounds';
    window.CURRENT_URL = encodeURIComponent(
        "".concat(
            window.location.protocol,
            "//",
            window.location.host,
            window.location.pathname,
            window.location.search
        )
    );
    window.FINANCE = {};
    window.FINANCE.BASE_URL = '/dist';
    window.FINANCE.VERSION = '1552461157';
    window.FINANCE.API_URL = '';
    window.FINANCE.HOST = '//finance.daum.net';
    window._REALTIME_HOST = 'wss://realtime-front-df.dunamu.com/websock